In [4]:
import plotly.graph_objects as go
from plotly import subplots
import pandas as pd
import numpy as np
from datetime import datetime
import refinitiv.data as rd
rd.open_session()


<refinitiv.data.session.Definition object at 0x16a668370 {name='workspace'}>

In [5]:
direct_qm = ['AUD = ', 'BTC = ', 'BWP = ', 'EUR = ', 'FJD = ', 'FKP = ', 'GBP = ', 'GIP = ', 'IEP = ', 'MTL = ',
             'NZD = ', 'PGK = ', 'SBD = ', 'SHP = ', 'TOP = ', 'USD = ', 'WST = ', 'XAG = ', 'XAU = ', 'XPD = ', 'XPT = ']

In [6]:
conf_int = 0.95
z_score_const = 1.64485


def convert_to_usd(ric, rate, amt):
    if ric in direct_qm:
        return amt * rate
    else:
        if rate != 0:
            return amt / rate
        else:
            return 0


def get_data(portfolio):
    instruments = list(portfolio.keys())
    df = rd.get_history(instruments, 'BID', interval='daily', count=252)
    df = df.fillna(method='ffill')
    positions = list(map(lambda p: portfolio[p][0], instruments))
    hedge_ratios = list(map(lambda p: portfolio[p][1], instruments))
    spot_fx = df.tail(1).values.tolist()[0]
    flow_usd = list(map(lambda r, f, a: convert_to_usd(
        r, f, a), instruments, spot_fx, positions))
    risk_amt_before_hedge = round(sum(flow_usd), 2)
    risk_pct_before_hedge = list(np.array(flow_usd) / risk_amt_before_hedge)
    hedge_usd = list(np.multiply(hedge_ratios, flow_usd))
    remaining_flow_usd = list(np.subtract(
        np.array(flow_usd), np.array(hedge_usd)))
    risk_amt_after_hedge = round(sum(remaining_flow_usd), 2)
    risk_pct_after_hedge = list(
        np.array(remaining_flow_usd) / risk_amt_after_hedge)

    # calculate log returns
    df_log = pd.DataFrame()
    df_log = np.log(df) - np.log(df.shift(1))
    df_log = df_log.dropna(axis=0)
    df_stats = df_log.describe()
    daily_std = list(map(lambda c: df_stats[c]['std'], list(df_stats.columns)))
    daily_avg = list(
        map(lambda c: df_stats[c]['mean'], list(df_stats.columns)))
    annualized_std = list(map(lambda x: x * (252 ** 0.5), daily_std))
    # calculate excess returns
    df_x_ret = df_log
    n = 0
    for c in list(df_x_ret.columns):
        df_x_ret[c] -= daily_avg[n]
        n += 1

    ann_std_hedged = list(np.multiply(
        np.array(risk_pct_after_hedge), np.array(annualized_std)))
    ann_std_unhedged = list(np.multiply(
        np.array(risk_pct_before_hedge), np.array(annualized_std)))

    # create correlation matrix on excess returns & vols
    corr_mtx = df_x_ret.corr()
    _vhedge = pd.DataFrame(corr_mtx.dot(ann_std_hedged)).transpose()
    _vunhedge = pd.DataFrame(corr_mtx.dot(ann_std_unhedged)).transpose()
    vol_after_hedge = np.dot(np.array(_vhedge), np.array(
        pd.DataFrame(ann_std_hedged)))[0][0] ** 0.5
    vol_before_hedge = np.dot(np.array(_vunhedge), np.array(
        pd.DataFrame(ann_std_unhedged)))[0][0] ** 0.5
    var_before_hedge = (vol_before_hedge *
                        np.abs(risk_amt_before_hedge) * z_score_const).round(2)
    var_after_hedge = (vol_after_hedge *
                       np.abs(risk_amt_after_hedge) * z_score_const).round(2)
    df_pct_risk = pd.DataFrame()
    df_pct_risk['% Risk after hedge'] = risk_pct_after_hedge
    df_pct_risk['% Risk before hedge'] = risk_pct_before_hedge
    df_pct_risk.index = instruments
    value_at_risk = pd.DataFrame()
    value_at_risk['Risk amount, USD'] = [
        risk_amt_before_hedge, risk_amt_after_hedge]
    value_at_risk['Volatility'] = [vol_before_hedge, vol_after_hedge]
    value_at_risk['VaR, USD'] = [var_before_hedge, var_after_hedge]
    value_at_risk.index = ['Before hedge', 'After hedge']
    trace1 = go.Heatmap(x=df_pct_risk.index, y=df_pct_risk.columns,
                        z=df_pct_risk.T.values * 100, colorscale='YlGn')
    trace2 = go.Bar(x=value_at_risk.index,
                    y=value_at_risk['VaR, USD'].values, marker=dict(color='#6978F7'))
    fig = subplots.make_subplots(shared_yaxes=False, shared_xaxes=False, rows=1, cols=2,
                                 subplot_titles=('VaR chg, USD', 'Risk distribution chg, %'), print_grid=False)
    fig.append_trace(trace1, row=1, col=2)
    fig.append_trace(trace2, row=1, col=1)
    fig.layout.title = 'FX hedge impact analysis'
    fig.layout.xaxis = dict(domain=[0, 0.25])
    fig.layout.xaxis2 = dict(domain=[0.45, 1])
    fig.update_layout(template='plotly_dark')

    return fig

In [ ]:
rd.close_session()
